In [521]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

state: US state, district, or territory.
declaration_type: One of "DR" (= major disaster), "EM" (= emergency management), or "FM" (= "fire management")
declaration_date: Date the disaster was declared.
fy_declared: Fiscal year in which the disaster was declared.
incident_type: Type of incident such as "Fire", "Flood", or "Hurricane". The incident type will affect the types of assistance available.
declaration_title: Title for the disaster. This can be a useful identifier such as "Hurricane Katrina" or "Covid-19 Pandemic".

In [523]:
data=pd.read_csv('C:/Users/Phantom/Documents/AI/FloodGuard/us_disaster_declarations.csv')
data.tail()

,fema_declaration_string,disaster_number,state,declaration_type,declaration_date,fy_declared,incident_type,declaration_title,ih_program_declared,ia_program_declared,...,incident_end_date,disaster_closeout_date,fips,place_code,designated_area,declaration_request_number,last_ia_filing_date,last_refresh,hash,id
64087,DR-4696-ME,4696,ME,DR,2023-03-22T00:00:00Z,2023,Severe Storm,Severe Storm And Flooding,0,0,...,2022-12-24T00:00:00Z,NaN,23031,99031,"York (County)(in MSA 6400,6450)",23013,NaN,2023-03-23T22:31:41Z,c6ed4ead7fd3bee8113ef15d9110e845c56a8b2f,3bd84f66-74d0-4e4a-9287-2414fda89014
64088,DR-4697-MS,4697,MS,DR,2023-03-26T00:00:00Z,2023,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",1,0,...,2023-03-25T00:00:00Z,NaN,28015,99015,Carroll (County),23025,2023-05-25T00:00:00Z,2023-03-26T06:41:23Z,9ccd8b25b2e64d4e44a043b1da61f3c42e2d02a9,764fa617-efd9-44c4-9d4a-c48e972b485c
64089,DR-4697-MS,4697,MS,DR,2023-03-26T00:00:00Z,2023,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",1,0,...,2023-03-25T00:00:00Z,NaN,28053,99053,Humphreys (County),23025,2023-05-25T00:00:00Z,2023-03-26T06:41:23Z,62d6f324279680dbb6b1eb2ce91a9bf5ae964e24,c9300696-062b-430f-9f9d-3ad6b1562779
64090,DR-4697-MS,4697,MS,DR,2023-03-26T00:00:00Z,2023,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",1,0,...,2023-03-25T00:00:00Z,NaN,28095,99095,Monroe (County),23025,2023-05-25T00:00:00Z,2023-03-26T06:41:23Z,5cb6b2789fc77375f0a1d0e1a8898a1ce3dfc952,df697ed2-4c59-4d02-84e7-d4e84d6e2723
64091,DR-4697-MS,4697,MS,DR,2023-03-26T00:00:00Z,2023,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",1,0,...,2023-03-25T00:00:00Z,NaN,28125,99125,Sharkey (County),23025,2023-05-25T00:00:00Z,2023-03-26T06:41:23Z,ef1b370fa2c218a02e815935fc7a35173f266e30,821952eb-f0a2-4109-9fa3-b7029e882874


In [524]:
data=data.drop(["fema_declaration_string","disaster_number","pa_program_declared","hm_program_declared","incident_begin_date","incident_end_date","fips","declaration_request_number","ih_program_declared","ia_program_declared","place_code","last_ia_filing_date","last_refresh","hash","id","disaster_closeout_date"],axis=1)
data.head()

,state,declaration_type,declaration_date,fy_declared,incident_type,declaration_title,designated_area
0,GA,DR,1953-05-02T00:00:00Z,1953,Tornado,Tornado,Statewide
1,TX,DR,1953-05-15T00:00:00Z,1953,Tornado,Tornado & Heavy Rainfall,Statewide
2,LA,DR,1953-05-29T00:00:00Z,1953,Flood,Flood,Statewide
3,MI,DR,1953-06-02T00:00:00Z,1953,Tornado,Tornado,Statewide
4,MT,DR,1953-06-06T00:00:00Z,1953,Flood,Floods,Statewide


In [525]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64092 entries, 0 to 64091
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   state              64092 non-null  object
 1   declaration_type   64092 non-null  object
 2   declaration_date   64092 non-null  object
 3   fy_declared        64092 non-null  int64 
 4   incident_type      64092 non-null  object
 5   declaration_title  64092 non-null  object
 6   designated_area    64092 non-null  object
dtypes: int64(1), object(6)
memory usage: 3.4+ MB


In [526]:
data['declaration_date']=data['declaration_date'].apply(lambda x: x[:10])

In [527]:
data.head()

,state,declaration_type,declaration_date,fy_declared,incident_type,declaration_title,designated_area
0,GA,DR,1953-05-02,1953,Tornado,Tornado,Statewide
1,TX,DR,1953-05-15,1953,Tornado,Tornado & Heavy Rainfall,Statewide
2,LA,DR,1953-05-29,1953,Flood,Flood,Statewide
3,MI,DR,1953-06-02,1953,Tornado,Tornado,Statewide
4,MT,DR,1953-06-06,1953,Flood,Floods,Statewide


In [528]:
data['incident_type'].nunique()

25

In [529]:
data['incident_type'].unique()

array(['Tornado', 'Flood', 'Fire', 'Other', 'Earthquake', 'Hurricane',
       'Volcanic Eruption', 'Severe Storm', 'Toxic Substances', 'Typhoon',
       'Drought', 'Dam/Levee Break', 'Snowstorm', 'Severe Ice Storm',
       'Freezing', 'Coastal Storm', 'Fishing Losses', 'Mud/Landslide',
       'Human Cause', 'Terrorist', 'Tsunami', 'Chemical', 'Biological',
       'Tropical Storm', 'Winter Storm'], dtype=object)

In [530]:
data.tail(10)

,state,declaration_type,declaration_date,fy_declared,incident_type,declaration_title,designated_area
64082,ME,DR,2023-03-22,2023,Severe Storm,Severe Storm And Flooding,Franklin (County)
64083,ME,DR,2023-03-22,2023,Severe Storm,Severe Storm And Flooding,Knox (County)
64084,ME,DR,2023-03-22,2023,Severe Storm,Severe Storm And Flooding,Oxford (County)
64085,ME,DR,2023-03-22,2023,Severe Storm,Severe Storm And Flooding,Somerset (County)
64086,ME,DR,2023-03-22,2023,Severe Storm,Severe Storm And Flooding,Waldo (County)
64087,ME,DR,2023-03-22,2023,Severe Storm,Severe Storm And Flooding,"York (County)(in MSA 6400,6450)"
64088,MS,DR,2023-03-26,2023,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Carroll (County)
64089,MS,DR,2023-03-26,2023,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Humphreys (County)
64090,MS,DR,2023-03-26,2023,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Monroe (County)
64091,MS,DR,2023-03-26,2023,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Sharkey (County)


In [531]:
data=data.rename(columns={'declaration_type': 'aloikatte','declaration_date':'date','fy_declared':'year','incident_type':'disaster','declaration_title':'disaster_info'})

In [532]:
data.head()

,state,aloikatte,date,year,disaster,disaster_info,designated_area
0,GA,DR,1953-05-02,1953,Tornado,Tornado,Statewide
1,TX,DR,1953-05-15,1953,Tornado,Tornado & Heavy Rainfall,Statewide
2,LA,DR,1953-05-29,1953,Flood,Flood,Statewide
3,MI,DR,1953-06-02,1953,Tornado,Tornado,Statewide
4,MT,DR,1953-06-06,1953,Flood,Floods,Statewide


In [533]:
data=data.drop(['year'],axis=1)

In [534]:
data

,state,aloikatte,date,disaster,disaster_info,designated_area
0,GA,DR,1953-05-02,Tornado,Tornado,Statewide
1,TX,DR,1953-05-15,Tornado,Tornado & Heavy Rainfall,Statewide
2,LA,DR,1953-05-29,Flood,Flood,Statewide
3,MI,DR,1953-06-02,Tornado,Tornado,Statewide
4,MT,DR,1953-06-06,Flood,Floods,Statewide
...,...,...,...,...,...,...
64087,ME,DR,2023-03-22,Severe Storm,Severe Storm And Flooding,"York (County)(in MSA 6400,6450)"
64088,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Carroll (County)
64089,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Humphreys (County)
64090,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Monroe (County)


In [535]:
data=data.rename(columns={'aloikatte':'category'})

In [536]:
def check_severity(row):
    if 'severe' in row['disaster'].lower() and 'severe' in row['disaster_info'].lower():
        return 1
    elif ('severe' in row['disaster'].lower() or 'severe' in row['disaster_info'].lower()) and not ('severe' in row['disaster'].lower() and 'severe' in row['disaster_info'].lower()):
        return 0.5
    else:
        return 0

data['severity'] = data.apply(check_severity, axis=1)
data.tail(15)

,state,category,date,disaster,disaster_info,designated_area,severity
64077,VT,DR,2023-03-20,Severe Storm,Severe Storm And Flooding,Grand Isle (County),1.0
64078,VT,DR,2023-03-20,Severe Storm,Severe Storm And Flooding,Lamoille (County),1.0
64079,VT,DR,2023-03-20,Severe Storm,Severe Storm And Flooding,Orange (County),1.0
64080,VT,DR,2023-03-20,Severe Storm,Severe Storm And Flooding,Orleans (County),1.0
64081,VT,DR,2023-03-20,Severe Storm,Severe Storm And Flooding,Washington (County),1.0
64082,ME,DR,2023-03-22,Severe Storm,Severe Storm And Flooding,Franklin (County),1.0
64083,ME,DR,2023-03-22,Severe Storm,Severe Storm And Flooding,Knox (County),1.0
64084,ME,DR,2023-03-22,Severe Storm,Severe Storm And Flooding,Oxford (County),1.0
64085,ME,DR,2023-03-22,Severe Storm,Severe Storm And Flooding,Somerset (County),1.0
64086,ME,DR,2023-03-22,Severe Storm,Severe Storm And Flooding,Waldo (County),1.0


In [537]:
data = data.drop_duplicates(subset=['state', 'date', 'disaster','disaster_info','designated_area'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64014 entries, 0 to 64091
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   state            64014 non-null  object 
 1   category         64014 non-null  object 
 2   date             64014 non-null  object 
 3   disaster         64014 non-null  object 
 4   disaster_info    64014 non-null  object 
 5   designated_area  64014 non-null  object 
 6   severity         64014 non-null  float64
dtypes: float64(1), object(6)
memory usage: 3.9+ MB


In [538]:
data

,state,category,date,disaster,disaster_info,designated_area,severity
0,GA,DR,1953-05-02,Tornado,Tornado,Statewide,0.0
1,TX,DR,1953-05-15,Tornado,Tornado & Heavy Rainfall,Statewide,0.0
2,LA,DR,1953-05-29,Flood,Flood,Statewide,0.0
3,MI,DR,1953-06-02,Tornado,Tornado,Statewide,0.0
4,MT,DR,1953-06-06,Flood,Floods,Statewide,0.0
...,...,...,...,...,...,...,...
64087,ME,DR,2023-03-22,Severe Storm,Severe Storm And Flooding,"York (County)(in MSA 6400,6450)",1.0
64088,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Carroll (County),1.0
64089,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Humphreys (County),1.0
64090,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Monroe (County),1.0


In [539]:
data.reset_index(drop=True, inplace=True)

In [540]:
data['disaster'].unique()

array(['Tornado', 'Flood', 'Fire', 'Other', 'Earthquake', 'Hurricane',
       'Volcanic Eruption', 'Severe Storm', 'Toxic Substances', 'Typhoon',
       'Drought', 'Dam/Levee Break', 'Snowstorm', 'Severe Ice Storm',
       'Freezing', 'Coastal Storm', 'Fishing Losses', 'Mud/Landslide',
       'Human Cause', 'Terrorist', 'Tsunami', 'Chemical', 'Biological',
       'Tropical Storm', 'Winter Storm'], dtype=object)

In [541]:
data.tail()

,state,category,date,disaster,disaster_info,designated_area,severity
64009,ME,DR,2023-03-22,Severe Storm,Severe Storm And Flooding,"York (County)(in MSA 6400,6450)",1.0
64010,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Carroll (County),1.0
64011,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Humphreys (County),1.0
64012,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Monroe (County),1.0
64013,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Sharkey (County),1.0


In [542]:
data=data[~data['disaster'].isin(['Other', 'Chemical', 'Biological', 
                                       'Fishing Losses', 'Dam/Levee Break', 
                                       'Terrorist', 'Human Cause', 
                                       'Toxic Substances'])]
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55839 entries, 0 to 64013
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   state            55839 non-null  object 
 1   category         55839 non-null  object 
 2   date             55839 non-null  object 
 3   disaster         55839 non-null  object 
 4   disaster_info    55839 non-null  object 
 5   designated_area  55839 non-null  object 
 6   severity         55839 non-null  float64
dtypes: float64(1), object(6)
memory usage: 3.4+ MB


In [543]:
data.loc[:, 'date'] = pd.to_datetime(data['date'])

# Filter for disasters from the year 2000 onward
data = data[data['date'] >= pd.Timestamp('2000-01-01')].copy()

# Show DataFrame info
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36230 entries, 19683 to 64013
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   state            36230 non-null  object 
 1   category         36230 non-null  object 
 2   date             36230 non-null  object 
 3   disaster         36230 non-null  object 
 4   disaster_info    36230 non-null  object 
 5   designated_area  36230 non-null  object 
 6   severity         36230 non-null  float64
dtypes: float64(1), object(6)
memory usage: 2.2+ MB


In [544]:
data

,state,category,date,disaster,disaster_info,designated_area,severity
19683,TX,FM,2000-01-03 00:00:00,Fire,Saddleback Fire,Wise (County),0.0
19684,TX,FM,2000-01-04 00:00:00,Fire,Purgatory Fire,Comal (County),0.0
19685,TX,FM,2000-01-05 00:00:00,Fire,Bob's Trail Fire,Bastrop (County),0.0
19686,KY,DR,2000-01-10 00:00:00,Tornado,"Tornadoes, Severe Storms, Torrential Rains, An...",Ballard (County),0.5
19687,KY,DR,2000-01-10 00:00:00,Tornado,"Tornadoes, Severe Storms, Torrential Rains, An...",Breckinridge (County),0.5
...,...,...,...,...,...,...,...
64009,ME,DR,2023-03-22 00:00:00,Severe Storm,Severe Storm And Flooding,"York (County)(in MSA 6400,6450)",1.0
64010,MS,DR,2023-03-26 00:00:00,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Carroll (County),1.0
64011,MS,DR,2023-03-26 00:00:00,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Humphreys (County),1.0
64012,MS,DR,2023-03-26 00:00:00,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Monroe (County),1.0


In [545]:
data.loc[:,'date'] = data['date'].astype(str)
data.loc[:,'date']=data['date'].apply(lambda x: x[:10])

In [546]:
data.tail()

,state,category,date,disaster,disaster_info,designated_area,severity
64009,ME,DR,2023-03-22,Severe Storm,Severe Storm And Flooding,"York (County)(in MSA 6400,6450)",1.0
64010,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Carroll (County),1.0
64011,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Humphreys (County),1.0
64012,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Monroe (County),1.0
64013,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Sharkey (County),1.0


In [547]:
data.reset_index(drop=True, inplace=True)
data

,state,category,date,disaster,disaster_info,designated_area,severity
0,TX,FM,2000-01-03,Fire,Saddleback Fire,Wise (County),0.0
1,TX,FM,2000-01-04,Fire,Purgatory Fire,Comal (County),0.0
2,TX,FM,2000-01-05,Fire,Bob's Trail Fire,Bastrop (County),0.0
3,KY,DR,2000-01-10,Tornado,"Tornadoes, Severe Storms, Torrential Rains, An...",Ballard (County),0.5
4,KY,DR,2000-01-10,Tornado,"Tornadoes, Severe Storms, Torrential Rains, An...",Breckinridge (County),0.5
...,...,...,...,...,...,...,...
36225,ME,DR,2023-03-22,Severe Storm,Severe Storm And Flooding,"York (County)(in MSA 6400,6450)",1.0
36226,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Carroll (County),1.0
36227,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Humphreys (County),1.0
36228,MS,DR,2023-03-26,Severe Storm,"Severe Storms, Straight-Line Winds, And Tornadoes",Monroe (County),1.0
